In [ ]:
# doc:
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition/client/start_label_detection.html#

    Use a UUID (Universally Unique Identifier) to generate a unique token.
    Ensure that this token is unique for each request to avoid unintended idempotency.

In [ ]:
You can generate a UUID in Python using the uuid library:

In [3]:
import uuid

client_request_token = str(uuid.uuid4())
client_request_token

'e4de56b4-6e9b-49ab-815e-16aaa5975b26'

In [5]:
import boto3
import json
import time

# Initialize clients
rekognition_client = boto3.client('rekognition')
sqs_client = boto3.client('sqs')

# Configuration
bucket_name = 'vivekp'
video_file = 'nature.mp4'
sns_topic_arn = 'arn:aws:sns:ap-south-1:381492233747:shot_detect'
iam_role_arn = 'arn:aws:iam::381492233747:role/vivek_role'
sqs_queue_url = 'https://sqs.ap-south-1.amazonaws.com/381492233747/shot_detection'

# Start label detection
response = rekognition_client.start_label_detection(
    Video={'S3Object': {'Bucket': bucket_name, 'Name': video_file}},
    NotificationChannel={'SNSTopicArn': sns_topic_arn, 'RoleArn': iam_role_arn},
    JobTag='DetectingLabels'
)
job_id = response['JobId']
print(f'Started label detection job with ID: {job_id}')


print(response)

Started label detection job with ID: 8b5ef1a219c6f1da2d4d008b23daf2d88ce440df38e003e4d490c1170463ef4d
{'JobId': '8b5ef1a219c6f1da2d4d008b23daf2d88ce440df38e003e4d490c1170463ef4d', 'ResponseMetadata': {'RequestId': '7e0af527-9754-4ac3-8962-37e2f889c461', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '7e0af527-9754-4ac3-8962-37e2f889c461', 'content-type': 'application/x-amz-json-1.1', 'content-length': '76', 'date': 'Thu, 23 May 2024 09:02:16 GMT'}, 'RetryAttempts': 0}}


#doc:

https://docs.aws.amazon.com/rekognition/latest/dg/segments.html?pg=ln&sec=ft#segment-shot-detection

In [6]:
#reference : https://github.com/Tom-Gray/awsPython/tree/master

import json
import os
import urllib

import boto3

def start_label_detection(bucket, key):
    rekognition_client = boto3.client('rekognition')
    response = rekognition_client.start_label_detection(
        Video={
            'S3Object': {
                'Bucket': 'vivekp',
                'Name': 'nature.mp4'
            }
        },
        NotificationChannel={
            'SNSTopicArn': 'arn:aws:sns:ap-south-1:381492233747:shot_detect',
            'RoleArn': 'arn:aws:iam::381492233747:role/vivek_role'
        })

    print(response)

    # while not done:
    #   time.sleep(10)
    #   done = check_job_status()

    return

def get_video_labels(job_id):
    rekognition_client = boto3.client('rekognition')

    response = rekognition_client.get_label_detection(JobId=job_id)

    next_token = response.get('NextToken', None)

    while next_token:
        next_page = rekognition_client.get_label_detection(
            JobId=job_id,
            NextToken=next_token
        )

        next_token = next_page.get('NextToken', None)

        response['Labels'].extend(next_page['Labels'])

    return response

def put_labels_in_db(data, video_name, video_bucket):
    del data["ResponseMetadata"]
    del data["JobStatus"]

    data["VideoName"] = video_name
    data["VideoBucket"] = video_bucket

    dynamodb = boto3.resource('dynamodb')
    table_name = os.environ['DYNAMO_DB_TABLE_NAME']
    videos_table = dynamodb.Table(table_name)

    videos_table.put_item(Item=data)
# Lambda events

def start_processing_video(event, context):
    for record in event['Records']:
        start_label_detection(
            record['s3']['bucket']['name'],
            urllib.parse.unquote_plus(record['s3']['object']['key'])
        )


    return

def handle_label_detection(event, context):
    for record in event['Records']:
        message = json.loads(record['Sns']['Message'])
        job_id = message['JobId']
        s3_object = message['Video']['S3ObjectName']
        s3_bucket = message['Video']['S3Bucket']

        response = get_video_labels(job_id)
        print(response)
        put_labels_in_db(response, s3_object, s3_bucket)

    return

# Sample event for start_processing_video function
sample_s3_event = {
    'Records': [{
        's3': {
            'bucket': {'name': 'your-bucket-name'},
            'object': {'key': 'your-video-file.mp4'}
        }
    }]
}

# Uncomment the lines below to test the functions locally
start_processing_video(sample_s3_event, None)
handle_label_detection(sample_sns_event, None)